In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__) 

TensorFlow version: 1.13.1


In [1]:
use_device='/gpu:0'


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Input, Concatenate
from tensorflow.keras.layers import LSTM

from tensorflow.python.client import device_lib

#import models and layers from tensorflow.keras to avoid errors like "TypeError: Checkpointable._track_checkpointable()"
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dropout, Activation, Flatten
from keras.optimizers import Adam
    
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#import tensorflow.keras.backend as K
from keras import regularizers
    
import pickle
import pandas as pd

from datetime import datetime

from sklearn.model_selection import train_test_split
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4785279148276276002
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3072013108
locality {
  bus_id: 1
  links {
  }
}
incarnation: 413357129044773927
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
# define paths and functions for loading models, define names, etc
# ========================================================================================================================   
path="xxx" #where to store the results (main path)
path_data= "xxx" #where is the dataset
path_data_2= "xxx"



X1_import=path_data + "SMILES_Tg.txt" #where is the Xdata of the dataset (SMILES)
Y_import=path_data + "Tg.txt" #where is the Ydata of the dataset (property to predict)
   

# Load SMILES strings from file
with open(X1_import, 'r') as f:
    smiles_data_1 = f.readlines()

# Load properties from file
with open(Y_import, 'r') as f:
    X_data = f.readlines()
    
print(smiles_data_1[10])
# print(smiles_data_2[10])
print(X_data[10])

OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@H]1O

311.15



In [4]:
# Split the dataset into train and test sets

# Preprocess SMILES data for SMILES_1

# Load the tokenizer
tokenizer_path = path_data_2 + 'tokenizer_all.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer_all = pickle.load(f)


sequences_1 = tokenizer_all.texts_to_sequences(smiles_data_1)
max_length_1 = max(len(seq) for seq in sequences_1)
padded_sequences_1 = pad_sequences(sequences_1, maxlen=max_length_1, padding='post')
print(np.shape(padded_sequences_1))


# Convert property data to numpy array
X_labels = np.array([float(prop.strip()) for prop in X_data])

# Define vocabulary size for SMILES_1
vocab_size_1 = len(tokenizer_all.word_index) + 1


# Define embedding dimension
embedding_dim = 128

# Split the dataset into train and test sets
test_size = 0.9
random_state = 42
X_train_1, X_test_1, y_train, y_test = train_test_split(
    padded_sequences_1, X_labels, test_size=test_size, random_state=random_state)
print(np.shape(X_train_1))
# print(np.shape(X_train_2))
print(np.shape(X_test_1))
# print(np.shape(X_test_2))

(501, 105)
(50, 105)
(451, 105)


In [5]:
#save X_test_1 and y_test to .npy file

np.save(path_data_2 + 'X_test_1.npy', X_test_1)
np.save(path_data_2 + 'y_test.npy', y_test)

In [6]:
from keras.callbacks import Callback
   
    
# checkpoint_filepath = path_data_2 + "Weights.npy" #'checkpoint'
monitor_custom='val_mean_absolute_percentage_error' #val_mean_absolute_error   val_mean_absolute_percentage_error
mode_custom='min'


early_stop= tf.keras.callbacks.EarlyStopping(monitor=monitor_custom,  verbose=3,
                                             mode=mode_custom, patience=500, restore_best_weights=True)
# callback_list= [model_checkpoint_callback, reduce_lr, early_stop]
callback_list= [early_stop]




def Model_embeddings(X_train_1, y_train, vocab_size_1, embedding_dim, max_length_1,
                     ep, b_s, v_s, shuffle_dataset, alpha, NeuronsFC, w1, w2, w3, channels_1, channels_2, 
                     channels_3, lr, beta_1, beta_2, epsilon, decay, amsgrad):

    
    # Create input shapes
    input_shape1 = X_train_1.shape[1:]
    # Define inputs
    A_input = Input(shape=input_shape1, name='INPUT')

    merged = Dense(NeuronsFC)(A_input)#(concatenated)
    merged = tf.keras.layers.LeakyReLU(alpha)(merged)
    output_layer = Dense(1, activation='linear')(merged)
    # Create model
    model = Model(inputs=[A_input], outputs=output_layer)
    print(model.summary(line_length=None, positions=None, print_fn=None))
    model_acts = Model(inputs=[A_input], outputs=merged)
    # Compile model
    opt = tf.keras.optimizers.Adam(lr, beta_1, beta_2, epsilon, decay, amsgrad)
    model.compile(loss='mean_absolute_percentage_error', optimizer=opt,
                           metrics=['mean_absolute_error', 'mean_absolute_percentage_error'])
    Data_X=[X_train_1]
    # Train the model
    History = model.fit(Data_X, y_train, epochs=ep, batch_size=b_s, validation_split=v_s,
              shuffle=shuffle_dataset, callbacks=callback_list)
    
    return model, model_acts, History

In [7]:
#DATA=====================================================================================================
Mode=0
#NUMBER OF TEST===========================================================================================
number_of_test = 1
#FREEZE_LAYERS============================================================================================
is_trainable_conv1_1=True #
is_trainable_conv1_2=True #
is_trainable_conv1_3=True #
is_trainable_conv1_4=True #
is_trainable_conv1_5=True
is_trainable_HIDDEN_A1_0=True #the CNN input side of the ANN
is_trainable_HIDDEN_A2_0=True #the Hbonds input side of the ANN (1:True, 2:True, 2:False)
is_trainable_HIDDEN_FC_all=True #concatenated FCs
#HYPERPARAMETERS==========================================================================================
lr=[0.1, 0.01, 0.001, 0.0001] #0.01
beta_1=0.99
beta_2=0.999
epsilon=None
decay=0
amsgrad=True
b_s=[1000, 20, 10, 5] #100
samples_on_control=0
samples_on_test=200
v_s= 0.1 #(samples_on_test+samples_on_control)/number_of_samples_X   
shuffle_dataset="False"
Drop_prob=0.2
alpha=0.05
reg=0.01
#WORKING PARAMETERS======================================================================================
number_epochs=6000 #
#FULLY CONNECTED=========================================================================================
NeuronsFC=50 # neurons combined FC
NeuronsFC_A2=5 #neurons in the Hbonds input
NeuronsFC_A1=5 #neurons in the CNN input
#CNN=====================================================================================================
w1=1 # must be the same if the model has been pre-trained
w2=6
w3=10
channels_1=32
channels_2=64
channels_3=128

In [10]:
# Iterate over each combination of learning rate and batch size

counter=0
Results=[]
for i in b_s:
    for j in lr:
        Name_saving="Tg_embedding_" + str(counter) + str(number_of_test)
        print("model name (to save): " + Name_saving)
        MODEL, ACTIVATIONS, HISTORY = Model_embeddings(X_train_1, y_train, vocab_size_1, embedding_dim, max_length_1,
                     number_epochs, i, v_s, shuffle_dataset, alpha, NeuronsFC, w1, w2, w3, channels_1, channels_2, 
                     channels_3, j, beta_1, beta_2, epsilon, decay, amsgrad)
        #save model
        try:
            MODEL.save(path_data_2 + Name_saving + ".model")
            print("Model saved")
        except:
            print("Model saved?")
            try:
                MODEL.save(path_data_2 + Name_saving + ".model")
                print("Model now saved!")
            except Exception as e:
                print("Error saving model:", e)
        #save acts
        try:
            ACTIVATIONS.save(path_data_2 + Name_saving + "_acts.model")
            print("Acts saved")
        except:
            print("Acts saved?")
            try:
                ACTIVATIONS.save(path_data_2 + Name_saving + "_acts.model")
                print("Acts now saved!")
            except Exception as e:
                print("Error saving model:", e)
        # Extract the mean absolute error from the training history
        mean_absolute_error_model = HISTORY.history['mean_absolute_error'][-1]  # Considering the last epoch error
        val_mean_absolute_error_model = HISTORY.history['val_mean_absolute_error'][-1]
        mean_absolute_percentage_error_model = HISTORY.history['mean_absolute_percentage_error'][-1]
        val_mean_absolute_percentage_error_model = HISTORY.history['val_mean_absolute_percentage_error'][-1]# Considering the last epoch_error_model
        # Append the results to the list
        Results.append([counter, j, i, mean_absolute_error_model, val_mean_absolute_error_model, mean_absolute_percentage_error_model, val_mean_absolute_percentage_error_model])            
        counter+=1

model name (to save): Tg_embedding_01
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
INPUT (InputLayer)           (None, 105)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)                5300      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 51        
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________
None
Train on 45 samples, validate on 5 samples
Epoch 1/6000
45/45 [==============================] - 1s 27ms/sample - loss: 100.9172 - mean_absolute_error: 249.1605 - mean_absolute_percentage_error: 100.9172 - val_loss: 65.2253 - val

Epoch 29/6000
45/45 [==============================] - 0s 445us/sample - loss: 34.5558 - mean_absolute_error: 79.0740 - mean_absolute_percentage_error: 34.5558 - val_loss: 35.6683 - val_mean_absolute_error: 103.9401 - val_mean_absolute_percentage_error: 35.6683
Epoch 30/6000
45/45 [==============================] - 0s 444us/sample - loss: 32.2912 - mean_absolute_error: 73.5445 - mean_absolute_percentage_error: 32.2912 - val_loss: 36.8301 - val_mean_absolute_error: 107.6140 - val_mean_absolute_percentage_error: 36.8301
Epoch 31/6000
45/45 [==============================] - 0s 378us/sample - loss: 29.1748 - mean_absolute_error: 65.8393 - mean_absolute_percentage_error: 29.1748 - val_loss: 38.4922 - val_mean_absolute_error: 112.9160 - val_mean_absolute_percentage_error: 38.4922
Epoch 32/6000
45/45 [==============================] - 0s 467us/sample - loss: 26.2979 - mean_absolute_error: 59.2020 - mean_absolute_percentage_error: 26.2979 - val_loss: 40.4349 - val_mean_absolute_error: 119.137

Epoch 61/6000
45/45 [==============================] - 0s 289us/sample - loss: 22.6704 - mean_absolute_error: 48.1049 - mean_absolute_percentage_error: 22.6704 - val_loss: 43.8608 - val_mean_absolute_error: 131.7021 - val_mean_absolute_percentage_error: 43.8608
Epoch 62/6000
45/45 [==============================] - 0s 289us/sample - loss: 21.6947 - mean_absolute_error: 46.7042 - mean_absolute_percentage_error: 21.6947 - val_loss: 44.1526 - val_mean_absolute_error: 132.5308 - val_mean_absolute_percentage_error: 44.1526
Epoch 63/6000
45/45 [==============================] - 0s 267us/sample - loss: 19.9769 - mean_absolute_error: 43.7645 - mean_absolute_percentage_error: 19.9769 - val_loss: 44.6861 - val_mean_absolute_error: 134.1327 - val_mean_absolute_percentage_error: 44.6861
Epoch 64/6000
45/45 [==============================] - 0s 222us/sample - loss: 18.3305 - mean_absolute_error: 41.1625 - mean_absolute_percentage_error: 18.3305 - val_loss: 45.3547 - val_mean_absolute_error: 136.177

Epoch 93/6000
45/45 [==============================] - 0s 400us/sample - loss: 15.6405 - mean_absolute_error: 34.7843 - mean_absolute_percentage_error: 15.6405 - val_loss: 49.4565 - val_mean_absolute_error: 148.7798 - val_mean_absolute_percentage_error: 49.4565
Epoch 94/6000
45/45 [==============================] - 0s 267us/sample - loss: 15.3808 - mean_absolute_error: 34.7731 - mean_absolute_percentage_error: 15.3808 - val_loss: 49.6217 - val_mean_absolute_error: 149.2378 - val_mean_absolute_percentage_error: 49.6217
Epoch 95/6000
45/45 [==============================] - 0s 311us/sample - loss: 14.4885 - mean_absolute_error: 33.4041 - mean_absolute_percentage_error: 14.4885 - val_loss: 49.9562 - val_mean_absolute_error: 150.2413 - val_mean_absolute_percentage_error: 49.9562
Epoch 96/6000
45/45 [==============================] - 0s 244us/sample - loss: 13.8830 - mean_absolute_error: 30.9936 - mean_absolute_percentage_error: 13.8830 - val_loss: 50.4071 - val_mean_absolute_error: 151.626

Epoch 125/6000
45/45 [==============================] - 0s 244us/sample - loss: 9.4270 - mean_absolute_error: 19.4760 - mean_absolute_percentage_error: 9.4270 - val_loss: 51.6961 - val_mean_absolute_error: 155.2419 - val_mean_absolute_percentage_error: 51.6961
Epoch 126/6000
45/45 [==============================] - 0s 244us/sample - loss: 9.4811 - mean_absolute_error: 19.7507 - mean_absolute_percentage_error: 9.4811 - val_loss: 51.9697 - val_mean_absolute_error: 156.1714 - val_mean_absolute_percentage_error: 51.9697
Epoch 127/6000
45/45 [==============================] - 0s 311us/sample - loss: 8.9916 - mean_absolute_error: 18.9095 - mean_absolute_percentage_error: 8.9916 - val_loss: 52.1220 - val_mean_absolute_error: 156.7189 - val_mean_absolute_percentage_error: 52.1220
Epoch 128/6000
45/45 [==============================] - 0s 267us/sample - loss: 8.3881 - mean_absolute_error: 17.6482 - mean_absolute_percentage_error: 8.3881 - val_loss: 52.2840 - val_mean_absolute_error: 157.2851 - 

...

45/45 [==============================] - 0s 2ms/sample - loss: 32.8347 - mean_absolute_error: 66.5800 - mean_absolute_percentage_error: 32.8348 - val_loss: 41.4113 - val_mean_absolute_error: 122.1355 - val_mean_absolute_percentage_error: 41.4113
Epoch 562/6000
45/45 [==============================] - 0s 2ms/sample - loss: 32.8108 - mean_absolute_error: 66.5279 - mean_absolute_percentage_error: 32.8108 - val_loss: 41.4018 - val_mean_absolute_error: 122.1072 - val_mean_absolute_percentage_error: 41.4018
Epoch 563/6000
45/45 [==============================] - 0s 2ms/sample - loss: 32.7809 - mean_absolute_error: 66.4563 - mean_absolute_percentage_error: 32.7809 - val_loss: 41.3970 - val_mean_absolute_error: 122.0945 - val_mean_absolute_percentage_error: 41.3970
Epoch 564/6000
45/45 [==============================] - 0s 2ms/sample - loss: 32.7484 - mean_absolute_error: 66.3765 - mean_absolute_percentage_error: 32.7484 - val_loss: 41.3958 - val_mean_absolute_error: 122.0935 - val_mean_absolu

45/45 [==============================] - 0s 2ms/sample - loss: 31.1356 - mean_absolute_error: 62.5392 - mean_absolute_percentage_error: 31.1356 - val_loss: 40.9922 - val_mean_absolute_error: 120.8869 - val_mean_absolute_percentage_error: 40.9922
Epoch 624/6000
45/45 [==============================] - 0s 2ms/sample - loss: 31.1201 - mean_absolute_error: 62.4956 - mean_absolute_percentage_error: 31.1201 - val_loss: 40.9950 - val_mean_absolute_error: 120.8982 - val_mean_absolute_percentage_error: 40.9950
Epoch 625/6000
45/45 [==============================] - 0s 2ms/sample - loss: 31.0908 - mean_absolute_error: 62.4255 - mean_absolute_percentage_error: 31.0908 - val_loss: 40.9937 - val_mean_absolute_error: 120.8964 - val_mean_absolute_percentage_error: 40.9937
Epoch 626/6000
45/45 [==============================] - 0s 2ms/sample - loss: 31.0560 - mean_absolute_error: 62.3577 - mean_absolute_percentage_error: 31.0560 - val_loss: 40.9886 - val_mean_absolute_error: 120.8820 - val_mean_absolu

45/45 [==============================] - 0s 2ms/sample - loss: 29.5112 - mean_absolute_error: 58.6958 - mean_absolute_percentage_error: 29.5112 - val_loss: 41.1971 - val_mean_absolute_error: 121.6659 - val_mean_absolute_percentage_error: 41.1971
Epoch 686/6000
45/45 [==============================] - 0s 2ms/sample - loss: 29.4860 - mean_absolute_error: 58.6389 - mean_absolute_percentage_error: 29.4860 - val_loss: 41.2036 - val_mean_absolute_error: 121.6886 - val_mean_absolute_percentage_error: 41.2036
Epoch 687/6000
45/45 [==============================] - 0s 2ms/sample - loss: 29.4593 - mean_absolute_error: 58.5799 - mean_absolute_percentage_error: 29.4593 - val_loss: 41.2089 - val_mean_absolute_error: 121.7080 - val_mean_absolute_percentage_error: 41.2089
Epoch 688/6000
45/45 [==============================] - 0s 2ms/sample - loss: 29.4336 - mean_absolute_error: 58.5130 - mean_absolute_percentage_error: 29.4336 - val_loss: 41.2082 - val_mean_absolute_error: 121.7083 - val_mean_absolu

45/45 [==============================] - 0s 2ms/sample - loss: 27.9353 - mean_absolute_error: 54.9454 - mean_absolute_percentage_error: 27.9353 - val_loss: 41.4585 - val_mean_absolute_error: 122.6246 - val_mean_absolute_percentage_error: 41.4585
Epoch 748/6000
45/45 [==============================] - 0s 2ms/sample - loss: 27.9104 - mean_absolute_error: 54.9151 - mean_absolute_percentage_error: 27.9104 - val_loss: 41.4648 - val_mean_absolute_error: 122.6462 - val_mean_absolute_percentage_error: 41.4648
Epoch 749/6000
45/45 [==============================] - 0s 2ms/sample - loss: 27.8835 - mean_absolute_error: 54.8595 - mean_absolute_percentage_error: 27.8835 - val_loss: 41.4814 - val_mean_absolute_error: 122.7014 - val_mean_absolute_percentage_error: 41.4814
Epoch 750/6000
45/45 [==============================] - 0s 2ms/sample - loss: 27.8578 - mean_absolute_error: 54.8192 - mean_absolute_percentage_error: 27.8578 - val_loss: 41.4948 - val_mean_absolute_error: 122.7468 - val_mean_absolu

45/45 [==============================] - 0s 2ms/sample - loss: 26.7685 - mean_absolute_error: 52.4263 - mean_absolute_percentage_error: 26.7685 - val_loss: 41.8205 - val_mean_absolute_error: 123.8226 - val_mean_absolute_percentage_error: 41.8205
Epoch 810/6000
45/45 [==============================] - 0s 2ms/sample - loss: 26.7454 - mean_absolute_error: 52.3776 - mean_absolute_percentage_error: 26.7454 - val_loss: 41.8242 - val_mean_absolute_error: 123.8339 - val_mean_absolute_percentage_error: 41.8242
Epoch 811/6000
45/45 [==============================] - 0s 2ms/sample - loss: 26.7320 - mean_absolute_error: 52.3475 - mean_absolute_percentage_error: 26.7320 - val_loss: 41.8232 - val_mean_absolute_error: 123.8300 - val_mean_absolute_percentage_error: 41.8232
Epoch 812/6000
45/45 [==============================] - 0s 2ms/sample - loss: 26.7143 - mean_absolute_error: 52.3111 - mean_absolute_percentage_error: 26.7143 - val_loss: 41.8283 - val_mean_absolute_error: 123.8460 - val_mean_absolu

In [11]:
#save as excel
#===============

# Define column names for the Excel file
columns = ['Counter', 'lr', 'bs', 'MAE', 'val MAE', 'MAPE', 'val MAPE']
# Convert Results array to a DataFrame
Results_df = pd.DataFrame(Results, columns=columns)
writer = pd.ExcelWriter(path_data_2 + "Results_file_" + str(number_of_test) + ".xlsx", engine='xlsxwriter')
Results_df.to_excel(writer, sheet_name='Results_file_', index=False)
writer.save()

In [12]:
Results_df.head(20)

,Counter,lr,bs,MAE,val MAE,MAPE,val MAPE
0,0,0.1000,1000,3.466743,162.871750,1.868206,54.477283
1,1,0.0100,1000,17.600565,141.760406,8.174646,47.221355
2,2,0.0010,1000,44.980286,129.528824,22.376801,43.436165
3,3,0.0001,1000,85.761436,123.705849,40.326973,42.154602
4,4,0.1000,20,8.945900,171.244278,3.358425,57.069908
5,5,0.0100,20,5.732600,194.552734,2.956336,64.120682
6,6,0.0010,20,31.900524,139.798065,16.448004,47.084141
7,7,0.0001,20,67.040443,127.363968,33.069283,43.016479
8,8,0.1000,10,9.732496,164.465714,4.230185,55.138378
9,9,0.0100,10,5.062199,179.449371,3.217489,59.499504
